# Setting up an OpenCMISS-Iron problem from scratch - jupyter notebook

## Introduction

This tutorial demonstrates how to setup and solve a problem using OpenCMISS-Iron from scratch in python. For the purpose of this tutorial, we will be solving a Laplace problem. In mathematics and physics, Laplace's equation is a second-order partial differential equation named after Pierre-Simon Laplace who first studied its properties.

The code excerpts in this section can be run interactively by entering it directly into the Python interpreter, which can be started by running python or ipython in a terminal. The code shown here can also be downloaded as a jupyter notebook in the following [link](https://github.com/OpenCMISS-Examples/basics_tutorial/blob/master/src/python/laplace_equation.ipynb).

## Loading OpenCMISS-Iron library
In order to use OpenCMISS we have to first import the opencmiss.iron module from the opencmiss package. This initialises library with default values \[and seeds random generator with a new value - move this to a parallel tutorial - too much detail for now\].

In [1]:
# Intialise OpenCMISS-Iron.
from opencmiss.iron import iron

Assuming OpenCMISS has been correctly built with the Python bindings by
following the instructions in the programmer documentation, we can now access
all the OpenCMISS functions, classes and constants under the iron namespace.

The next section describes how we can interact with the OpenCMISS-Iron library
through an object-oriented API.

## OpenCMISS-Iron objects

### Creating objects

Every OpenCMISS-Iron object can be created using the following steps that are illustrated for creating a coordinate system object:


1. Create an instance of the class of interest from the iron module, ie `instance = iron.Class_name()`.

    `coordinate_system = iron.CoordinateSystem()`


2. Set default attributes of the object using the `CreateStart` method. This requires the user to provide an identifier that is unique for each object class. This identifier is typically termed a user number.

    `coordinate_system_user_number = 1`

    `coordinate_system.CreateStart(coordinate_system_user_number)`


3. Customise the attributes of the object by calling class methods.

    `coordinate_system.DimensionSet(3)`


4. Finalise the object. An object cannot be used until it is finalised.

    `coordinate_system.CreateFinish()`

### Destroying objects

Once finalised, the parameters of the object cannot typically be modified. In order to modify the object, either another object needs to be defined with a different user number, or the object needs to be destroyed as shown below:

`coordinate_system.Destroy()`

## Setting up model geometry

The first step in our example will be to set up a region and coordinate system and create a 3D mesh.

In [2]:
# Create coordinate system.
coordinate_system_user_number = 1
coordinate_system = iron.CoordinateSystem()
coordinate_system.CreateStart(coordinate_system_user_number)
coordinate_system.CreateFinish()

region_user_number = 1
region = iron.Region()
region.CreateStart(region_user_number, iron.WorldRegion)
region.CoordinateSystemSet(coordinate_system)
region.CreateFinish()

We then create basis functions for the mesh. By default, OpenCMISS-Iron will define a linear Lagrange basis. As we have 3 geometric coordinates, we will have a tri-linear Lagrange basis.

In [3]:
basis_user_number = 1
basis = iron.Basis()
basis.CreateStart(basis_user_number)
basis.CreateFinish()

In this example we will use the generated mesh capabilities of OpenCMISS to create a 3D geometric mesh on which to solve the Laplace problem. We will create a regular mesh of size width x height x length and divide the mesh into `number_global_x_elements` in the X direction,  `number_global_y_elements` in the Y direction and `number_global_z_elements` in the Z direction.

In [4]:
#  Define mesh parameters.
number_global_x_elements = 1
number_global_y_elements = 3
number_global_z_elements = 1
height = 1.0
width = 1.0
length = 1.0

# Create mesh using the iron.GeneratedMesh object.
generated_mesh_user_number = 1
generated_mesh = iron.GeneratedMesh()
generated_mesh.CreateStart(generated_mesh_user_number, region)
generated_mesh.TypeSet(iron.GeneratedMeshTypes.REGULAR)
generated_mesh.BasisSet([basis])
generated_mesh.ExtentSet([width, height, length])
generated_mesh.NumberOfElementsSet(
    [number_global_x_elements,
     number_global_y_elements,
     number_global_z_elements])

When we finish generating the mesh we have a mesh object returned to us. This mesh object is just the same as if we had manually created the regular mesh.

In [5]:
mesh_user_number = 1
mesh = iron.Mesh()
generated_mesh.CreateFinish(mesh_user_number,mesh)


Once the mesh has been created we can decompose it into a number of domains in order to allow for parallelism. We choose the options to let OpenCMISS calculate the best way to break up the mesh. We also set the number of domains to be equal to the number of computational nodes this example is running on. Note that if MPI infrastructure is not used, only single domain will ve created.

In [6]:
# Perform mesh decomposition.
decomposition_user_number = 1
decomposition = iron.Decomposition()
decomposition.CreateStart(decomposition_user_number, mesh)
decomposition.CreateFinish()

Now that the mesh has been decomposed we are in a position to create fields. The first field we need to create is the geometry field. Here we create a field and set the field’s mesh decomposition to the decomposed mesh that we have just created. We can choose exact how each component of the field is interpolated by setting component mesh component to be the mesh components that we created for the mesh. For this example we only have one mesh component. Once we have finished creating the field we can change the field DOFs to give us our geometry. Since this mesh has been generated we can use the generated mesh object to calculate the geometric parameters of the regular mesh.

In [7]:
# Create a field for the geometry.
geometric_field_user_number = 1
geometric_field = iron.Field()
geometric_field.CreateStart(geometric_field_user_number, region)
geometric_field.MeshDecompositionSet(decomposition)
geometric_field.CreateFinish()

In [8]:
# Set geometric field values from the generated mesh.
generated_mesh.GeometricParametersCalculate(geometric_field)

We are now in a position to define the type of physics that we wish to solve. This is done by creating an equations set which is a contianer object for all the parameters we need to describe the physics. Here we create a standard Laplace equations set.

$$\displaystyle \nabla ^{2}f={\frac {\partial ^{2}f}{\partial x^{2}}}+{\frac {\partial ^{2}f}{\partial y^{2}}}+{\frac {\partial ^{2}f}{\partial z^{2}}}=0.$$

In [9]:
# Create standard Laplace equations set.
equations_set_user_number = 1
equations_set_field_user_number = 2
equations_set_field = iron.Field()
equations_set = iron.EquationsSet()
equations_set_specification = [
    iron.EquationsSetClasses.CLASSICAL_FIELD,
    iron.EquationsSetTypes.LAPLACE_EQUATION,
    iron.EquationsSetSubtypes.STANDARD_LAPLACE]
equations_set.CreateStart(
    equations_set_user_number, region, geometric_field,
    equations_set_specification, equations_set_field_user_number,
    equations_set_field)
equations_set.CreateFinish()

For the Laplace equation we need a dependent field (our solution). Here we do not define a field before the create starts and so we let OpenCMISS create an appropriate dependent field for the Laplace equations being described. Once the fields have been created we can set the field DOF values.

In [10]:
# Create dependent field.
dependent_field_user_number = 3
dependent_field = iron.Field()
equations_set.DependentCreateStart(
    dependent_field_user_number, dependent_field)
equations_set.DependentCreateFinish()

# Initialise dependent field.
dependent_field.ComponentValuesInitialiseDP(
    iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES, 1, 0.5)

In [11]:
# Create equations.
equations = iron.Equations()
equations_set.EquationsCreateStart(equations)
equations_set.EquationsCreateFinish()

Now we are ready to set up a problem to be solved by OpenCMISS.

In [12]:
# Create problem.
problem_user_number = 1
problem = iron.Problem()
problem_specification = [
    iron.ProblemClasses.CLASSICAL_FIELD,
    iron.ProblemTypes.LAPLACE_EQUATION,
    iron.ProblemSubtypes.STANDARD_LAPLACE]
problem.CreateStart(problem_user_number, problem_specification)
problem.CreateFinish()

OpenCMISS control loop is a "supervisor" for the computational process.

In [13]:
# Create control loops.
problem.ControlLoopCreateStart()
problem.ControlLoopCreateFinish()

We are now ready to setup a solver for the problem. An iterative solver is used by default.

In [14]:
# Create problem solver
solver = iron.Solver()
problem.SolversCreateStart()
problem.SolverGet([iron.ControlLoopIdentifiers.NODE], 1, solver)
solver.OutputTypeSet(iron.SolverOutputTypes.SOLVER)
problem.SolversCreateFinish()

# Create solver equations and add equations set to solver equations.
solver = iron.Solver()
solver_equations = iron.SolverEquations()
problem.SolverEquationsCreateStart()
problem.SolverGet([iron.ControlLoopIdentifiers.NODE], 1, solver)
solver.SolverEquationsGet(solver_equations)
solver_equations.EquationsSetAdd(equations_set)
problem.SolverEquationsCreateFinish()

The Dirichlet problem for Laplace's equation consists of finding a solution φ on some domain D such that φ on the boundary of D is equal to some given function. Since the Laplace operator appears in the heat equation, one physical interpretation of this problem is as follows: fix the temperature on the boundary of the domain according to the given specification of the boundary condition. Allow heat to flow until a stationary state is reached in which the temperature at each point on the domain doesn't change anymore. The temperature distribution in the interior will then be given by the solution to the corresponding Dirichlet problem.

In [15]:
# Identify first and last node number.
firstNodeNumber = 1
nodes = iron.Nodes()
region.NodesGet(nodes)
lastNodeNumber = nodes.NumberOfNodesGet()

# Create boundary conditions and set first and last nodes to 0.0 and 1.0
boundary_conditions = iron.BoundaryConditions()
solver_equations.BoundaryConditionsCreateStart(boundary_conditions)
boundary_conditions.SetNode(
    dependent_field, iron.FieldVariableTypes.U, 1, 1, firstNodeNumber,
    1, iron.BoundaryConditionsTypes.FIXED, 0.0)
boundary_conditions.SetNode(
    dependent_field, iron.FieldVariableTypes.U, 1, 1, lastNodeNumber,
    1, iron.BoundaryConditionsTypes.FIXED, 1.0)
solver_equations.BoundaryConditionsCreateFinish()

# Solve the problem.
problem.Solve()

Now we want to have the results of the run be stored for analysis and later use

In [16]:
# Export results in FieldML format.
base_name = "laplace_equation"
data_format = "PLAIN_TEXT"

fml = iron.FieldMLIO()
fml.OutputCreate(mesh, "", base_name, data_format)
fml.OutputAddFieldNoType(
    base_name + ".geometric", data_format, geometric_field,
    iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES)
fml.OutputAddFieldNoType(
    base_name + ".phi", data_format, dependent_field,
    iron.FieldVariableTypes.U, iron.FieldParameterSetTypes.VALUES)
fml.OutputWrite("laplace_equation.xml")
fml.Finalise()

# Export results in Exfile format.
fields = iron.Fields()
fields.CreateRegion(region)
fields.NodesExport("laplace_equation", "FORTRAN")
fields.ElementsExport("laplace_equation", "FORTRAN")
fields.Finalise()

Let the library know that you are done with computations and the resources allocated for the problem can now be released


In [17]:
iron.Finalise()